In [1]:
#Configure your config.py on your local machine with your yelp and google API keys and add your config.py file to your .gitignore
#message on slack for questions

In [38]:
import csv
import pandas as pd
import requests
import json
from config import API_KEY, gkey
from pprint import pprint
import gmaps

In [3]:
pd.show_versions()



INSTALLED VERSIONS
------------------
commit: None
python: 3.6.6.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.4
pytest: 3.8.0
pip: 10.0.1
setuptools: 40.2.0
Cython: 0.28.5
numpy: 1.15.1
scipy: 1.1.0
pyarrow: None
xarray: None
IPython: 6.5.0
sphinx: 1.7.9
patsy: 0.5.0
dateutil: 2.7.3
pytz: 2018.5
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.8
feather: None
matplotlib: 2.2.2
openpyxl: 2.5.6
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.1.0
lxml: 4.2.3
bs4: 4.6.3
html5lib: 1.0.1
sqlalchemy: 1.2.11
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


In [2]:
df_beer_reviews=pd.read_csv('../beer_reviews.csv', encoding = "ISO-8859-1")

In [3]:
df_beer_reviews.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
beer = df_beer_reviews.groupby(["beer_name","brewery_name","beer_style"])
beer.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [5]:
breweries = list(df_beer_reviews['brewery_name'])
breweries
unique_breweries = set(breweries)
unique_breweries
unique_brew = df_beer_reviews['brewery_name'].unique()
len(unique_breweries)

5743

In [141]:
#https://stackoverflow.com/questions/43568760/pandas-drop-rows-where-column-contains
#https://stackoverflow.com/questions/28679930/how-to-drop-rows-from-pandas-data-frame-that-contains-a-particular-string-in-a-p/28680078
#https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
#https://stackoverflow.com/questions/41833624/delete-rows-based-on-a-condition-in-pandas
#https://stackoverflow.com/questions/8305199/the-tilde-operator-in-python
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html
#https://stackoverflow.com/questions/20731966/regex-remove-all-special-characters-except-numbers
unique_brew_df = pd.DataFrame(unique_brew)
clean_brew_df = unique_brew_df[~unique_brew_df[0].astype(str).str.contains('[^a-zA-Z0-9 ]', regex=True)]
clean_brew_df = clean_brew_df.reset_index()
clean_brew_df = clean_brew_df.drop(columns='index')
clean_brew_df.columns

Index([0], dtype='object')

In [145]:
clean_list = list(clean_brew_df[0])


In [148]:
 
put = "textquery"                               
field = "id,name,rating,geometry/location"
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
all_lat_list = []
all_lng_list = []
all_brew_list = []
all_rating_list = []
all_response_list = []
for brewery in clean_list:
    params = {
    "input": brewery,
    "key": gkey,
    "inputtype": put,
    "fields": field
}
    print(f"Making request for {brewery}.")
    all_response = requests.get(base_url, params=params)
    all_result = all_response.json()
    all_response_list.append(all_response.json())
    
    try:
        all_lat_list.append(all_result['candidates'][0]['geometry']['location']['lat'])
        all_lng_list.append(all_result['candidates'][0]['geometry']['location']['lng'])
        all_brew_list.append(all_result['candidates'][0]['name'])
        all_rating_list.append(all_result['candidates'][0]['rating'])
    except:
        print(f"No result for {brewery}.")
        all_lat_list.append('No Google Result')
        all_lng_list.append('No Google Result')
        all_brew_list.append(f'No Google Result for {brewery}')
        all_rating_list.append('No Google Result')
    



Making request for Vecchio Birraio.
Making request for Caldera Brewing Company.
Making request for Broad Ripple Brew Pub.
Making request for Moon River Brewing Company.
Making request for Kirner Bier.
No result for Kirner Bier.
Making request for Conwy Brewery Ltd.
Making request for Pacific Coast Brewing Company.
Making request for City Grille and Brewhaus.
No result for City Grille and Brewhaus.
Making request for San Francisco Brewing Company.
Making request for Yazoo Brewing Company.
Making request for 7 Seas Brewery and Taproom.
Making request for Calapooia Brewing Company.
Making request for Brouwerij Van Viven.
No result for Brouwerij Van Viven.
Making request for Destiny Brewing Company.
No result for Destiny Brewing Company.
Making request for Hokusetsu Brewery.
Making request for Bernard Family Brewery.
Making request for Isle Of Skye Brewery.
Making request for Brauhaus Lemke.
Making request for Founders Brewing Company.
No result for Founders Brewing Company.
Making request

Making request for Brauerei Ried.
Making request for Les Brasseurs de Gayant.
No result for Les Brasseurs de Gayant.
Making request for Craig Street Brew Pub.
Making request for Allendale Brew Company Limited.
No result for Allendale Brew Company Limited.
Making request for Robinsons Family Brewers.
No result for Robinsons Family Brewers.
Making request for Three Heads Brewing.
Making request for Alley Kat Brewing Company.
Making request for Brauerei Georg Michael GmbH.
No result for Brauerei Georg Michael GmbH.
Making request for Northshire Brewery.
Making request for Pivovar Rakovnik.
Making request for Bere Trei Stejari.
Making request for Grans Bryggeri AS.
Making request for Odd Side Ales.
Making request for Via Priula.
No result for Via Priula.
Making request for Pivovoar Branik.
No result for Pivovoar Branik.
Making request for Brouwerij West.
Making request for Pilsnergubbarna AB.
No result for Pilsnergubbarna AB.
Making request for Mallard Brewery.
No result for Mallard Brewer

Making request for Stallhagens Bryggeri Ab.
No result for Stallhagens Bryggeri Ab.
Making request for Shaws Brewery.
No result for Shaws Brewery.
Making request for West Coast Brewery.
Making request for Cotswold Brewing Company.
Making request for Chastnaya Pivovarnya Erstes.
No result for Chastnaya Pivovarnya Erstes.
Making request for Oldershaw Brewery.
Making request for Den Gale Brygger.
No result for Den Gale Brygger.
Making request for Big City Brewing CO LTD.
No result for Big City Brewing CO LTD.
Making request for Butterfield Brewing Company.
No result for Butterfield Brewing Company.
Making request for Bryggeriet Skands.
No result for Bryggeriet Skands.
Making request for Yellow Cross.
No result for Yellow Cross.
Making request for Pivovarinaya Kompaniya Utes.
No result for Pivovarinaya Kompaniya Utes.
Making request for Bar Teerenpeli.
No result for Bar Teerenpeli.
Making request for Zum Stiefel.
Making request for Sound Brewery.
Making request for Dreher Brewery.
No result

Making request for Notch Brewing.
Making request for Tun Tavern.
No result for Tun Tavern.
Making request for Crazy Mountain Brewing Company.
Making request for Brauerei Zur Sonne.
Making request for Brasserie Alauna.
No result for Brasserie Alauna.
Making request for Brauerei Franz Xaver Glossner.
Making request for Illinois Brewing Company.
Making request for Benderskiy Pivovarinniy Zavod.
No result for Benderskiy Pivovarinniy Zavod.
Making request for Kinn Bryggeri AS.
No result for Kinn Bryggeri AS.
Making request for Spectrum Brewery.
Making request for Mission Springs Brewing Company.
Making request for Birrificio Lambrate.
Making request for Brasserie Artisanale Millevertus De Toernich.
Making request for 2nd Shift Brewery.
Making request for Hollywood Beach Brewing Company.
Making request for Janake Wine Group AB.
Making request for Old Spot Brewery Ltd.
No result for Old Spot Brewery Ltd.
Making request for Brouwerij De Leckere.
No result for Brouwerij De Leckere.
Making reque

Making request for Brasserie de Rochefort.
No result for Brasserie de Rochefort.
Making request for Metropolitan Brewing.
Making request for Cropton Brewery.
Making request for Brasserie Du Canardou.
No result for Brasserie Du Canardou.
Making request for Les Brasseurs du Hameau.
No result for Les Brasseurs du Hameau.
Making request for Brouwerij De Cock.
Making request for Stevens Point Brewery.
Making request for Indian Wells Brewing Company.
Making request for Cervejaria Sul Brasileira.
No result for Cervejaria Sul Brasileira.
Making request for Brasserie Des Vignes.
Making request for Kern River Brewing Company.
Making request for Wylam Brewery.
Making request for Skull Coast Brewing Company.
Making request for Sheepscot Valley Brewing Company.
Making request for Bierbrouwerij Onder De Linden.
Making request for Clarens Brewery.
Making request for Oyster Bar Brewery.
No result for Oyster Bar Brewery.
Making request for Carlyle Brewing Company.
Making request for Bitter Creek Brewin

...

Making request for Bulgarhmel.
No result for Bulgarhmel.
Making request for Furthermore Beer.
No result for Furthermore Beer.
Making request for 3 Stars Brewing Company.
Making request for Garton Brewery.
No result for Garton Brewery.
Making request for Hochstiftliches Brauhaus Fulda GmbH.
Making request for Wharfebank Brewery.
Making request for Briar Creek Brewery.
No result for Briar Creek Brewery.
Making request for El Segundo Brewing Company.
Making request for Cashmere Brewing Company.
No result for Cashmere Brewing Company.
Making request for Shakespeare Tavern and Brewery.
Making request for Steinfels.
Making request for Rocky Mountain Brewing.
Making request for River House Brewpub.
Making request for Brewboys.
Making request for Borealis Brewery.
No result for Borealis Brewery.
Making request for Buntingford Brewery.
Making request for Barum Brewery Limited.
Making request for Brasserie Le Gambrinus.
Making request for San Pedro Brewing Company.
Making request for Anchor Brew

No result for DuClaw Brewing Company.
Making request for City Of Cambridge Brewery Company Limited.
No result for City Of Cambridge Brewery Company Limited.
Making request for Brasserie Val De Sambre.
Making request for Sudwerk Restaurant and Brewery.
Making request for Stonecutters Brewhouse.
No result for Stonecutters Brewhouse.
Making request for Phantom Canyon Brewing Company.
Making request for East Coast Beer Company.
No result for East Coast Beer Company.
Making request for Redhook Ale Brewery.
No result for Redhook Ale Brewery.
Making request for Shenzhen Kingway Brewery.
Making request for Das Bierhaus.
Making request for Privatbrauerei Glaab.
Making request for Hue Brewery LTD.
No result for Hue Brewery LTD.
Making request for Flat 12 Bierwerks.
No result for Flat 12 Bierwerks.
Making request for Thomas Creek Brewery.
Making request for North Fork Brewing.
Making request for Falstaff Brewing Company.
No result for Falstaff Brewing Company.
Making request for Tundra Brewery.
N

No result for Brauerei Stempfhuber.
Making request for Texelse Bierbrouwerij.
No result for Texelse Bierbrouwerij.
Making request for Oceanbryggeriet.
Making request for Small Town Brewery.
Making request for Les Brasseurs Qui Fument.
No result for Les Brasseurs Qui Fument.
Making request for Gore Range Brewery.
Making request for Northampton Brewery.
Making request for Okanagan Spring Brewery.
Making request for Kolberg Brauerei.
Making request for Brauerei Witzgall.
Making request for Dublin Brewing Company.
Making request for Brand Bierbrouwerij.
Making request for Tofino Brewing Company.
Making request for Statale Nove.
Making request for Old North State Winery And Brewery.
Making request for Museumbrouwerij De Roos.
Making request for Brasserie Rancho El Paso.
No result for Brasserie Rancho El Paso.
Making request for Konrad Krug Brauerei.
No result for Konrad Krug Brauerei.
Making request for Pivovar Kocour Varnsdorf.
Making request for Duckstein Brewery.
Making request for Ahlaf

No result for Persha Privatna Brovariya.
Making request for Brouwerij De Ryck.
Making request for Arctic Craft Brewery.
Making request for Seabright Brewery.
Making request for La Micro de Bromont.
Making request for Pilot Bay Brewing Company.
No result for Pilot Bay Brewing Company.
Making request for Weissbierbrauerei Simon KG.
No result for Weissbierbrauerei Simon KG.
Making request for Brauerei Stadtguet.
Making request for Luckie Ales.
Making request for Spitalbrauerei Regensburg.
Making request for Brauhaus Mitte.
Making request for Little River Brewing Company.
Making request for Gilgamesh Brewing.
Making request for San Luis Valley Brewing Company.
Making request for Highgate Brewery Ltd.
No result for Highgate Brewery Ltd.
Making request for Birrificio Lodigiano.
Making request for Grand Canyon Brewery.
Making request for Sutter Buttes Brewing.
Making request for Orkney Brewery.
Making request for Bitburger Brauerei.
No result for Bitburger Brauerei.
Making request for Coronad

Making request for Great White North Craft Brewery.
No result for Great White North Craft Brewery.
Making request for Browar Krotoszyn.
Making request for Panil.
No result for Panil.
Making request for Bayreuther Bierbrauerei.
Making request for Cannery Brewing Company.
Making request for De Bekeerde Suster.
Making request for Castle Springs Brewing Company.
No result for Castle Springs Brewing Company.
Making request for Willoughby Brewing Company.
Making request for Microbirrificio Opperbacco.
Making request for Factory Beer.
No result for Factory Beer.
Making request for Braumanufaktur Forsthaus Templin.
Making request for De Tacchi.
Making request for The Bronx Brewery.
Making request for Denen Plaza Kawaba Brewery.
Making request for The Famous Railway Tavern Brewery.
No result for The Famous Railway Tavern Brewery.
Making request for West Brothers Brewery.
Making request for Berghoeve Brouwerij.
Making request for Twisted X Brewing Company.
Making request for Broken Drum Brewery 

In [149]:
all_df = pd.DataFrame({'Brewery': all_brew_list, 'Latitude': all_lat_list, 'Longitude': all_lng_list, 'Rating': all_rating_list})
all_df = all_df.reset_index()
all_df

,index,Brewery,Latitude,Longitude,Rating
0,0,Birreria Il Vecchio Birraio,45.5509,11.8285,4.3
1,1,Caldera Brewery & Restaurant,42.1819,-122.663,4.5
2,2,Broad Ripple Brewpub,39.8736,-86.1429,4.4
3,3,Moon River Brewing Company,32.081,-81.092,3.9
4,4,No Google Result for Kirner Bier,No Google Result,No Google Result,No Google Result
5,5,Conwy Brewery Ltd,53.2837,-3.67631,4.5
6,6,Public Coast Brewing Co,45.9008,-123.959,4
7,7,No Google Result for City Grille and Brewhaus,No Google Result,No Google Result,No Google Result
8,8,San Francisco Brewing Co.,37.8059,-122.423,4.1
9,9,Yazoo Brewing Company,36.1512,-86.7821,4.5


In [154]:
clean_all_df = all_df[~all_df['Latitude'].astype(str).str.contains('No Google Result')]
clean_all_df = clean_all_df.reset_index()
clean_all_df = clean_all_df.drop(columns={'level_0', 'index'})
clean_all_df

,Brewery,Latitude,Longitude,Rating
0,Birreria Il Vecchio Birraio,45.5509,11.8285,4.3
1,Caldera Brewery & Restaurant,42.1819,-122.663,4.5
2,Broad Ripple Brewpub,39.8736,-86.1429,4.4
3,Moon River Brewing Company,32.081,-81.092,3.9
4,Conwy Brewery Ltd,53.2837,-3.67631,4.5
5,Public Coast Brewing Co,45.9008,-123.959,4
6,San Francisco Brewing Co.,37.8059,-122.423,4.1
7,Yazoo Brewing Company,36.1512,-86.7821,4.5
8,7 Seas Brewery and Taproom,47.2422,-122.439,4.6
9,Calapooia Brewing,44.6393,-123.094,4.3


In [ ]:
all_yelp_name_list = []
all_yelp_rating_list  = []
all_yelp_count_list = []
SEARCH_LIMIT = 1
category = "breweries"
sort = "distance"
all_yelp_response_list = []

    


for index,row in clean_all_df.iterrows():
    API_HOST = 'https://api.yelp.com'
    BUSINESS_PATH = '/v3/businesses/search'
    business_path = BUSINESS_PATH
    url = API_HOST + business_path
    headers = {'Authorization': f"Bearer {API_KEY}"}
    url_params = {
        'longitude': row['Longitude'],
        'latitude': row['Latitude'],
        'limit': SEARCH_LIMIT,
        'categories': category,
        'sort_by': sort
    }
    print(f"Making request for {row['Brewery']}, {row['Latitude']}, {row['Longitude']}.")
    all_yelp_response = requests.get(url, headers=headers, params=url_params)
    all_yelp_result = all_yelp_response.json()
    all_yelp_response_list.append(all_yelp_response.json())
    
    try:
        all_yelp_name_list.append(all_yelp_result['businesses'][0]['name'])
        all_yelp_rating_list.append(all_yelp_result['businesses'][0]['rating'])
        all_yelp_count_list.append(all_yelp_result['businesses'][0]['review_count'])
        
    except:
        print(f"No result for {row['Brewery']}.")
        all_yelp_name_list.append(f"No Yelp Result for {row['Brewery']}")
        all_yelp_rating_list.append("No Yelp Result")
        all_yelp_count_list.append("No Yelp Result")


Making request for Birreria Il Vecchio Birraio, 45.5509046, 11.8284632.
Making request for Caldera Brewery & Restaurant, 42.1819009, -122.662854.
Making request for Broad Ripple Brewpub, 39.8735658, -86.1428589.
Making request for Moon River Brewing Company, 32.080962, -81.09203.
Making request for Conwy Brewery Ltd, 53.2836814, -3.6763056.
Making request for Public Coast Brewing Co, 45.9007735, -123.9590522.
Making request for San Francisco Brewing Co., 37.8058697, -122.4230558.
Making request for Yazoo Brewing Company, 36.151225, -86.782068.
Making request for 7 Seas Brewery and Taproom, 47.2421769, -122.4388206.
Making request for Calapooia Brewing, 44.6392607, -123.0940026.
Making request for Hokusetsu Sake Brewery, 37.870728, 138.4136249.
No result for Hokusetsu Sake Brewery.
Making request for BERNARD Family Brewery Inc., 49.539955, 15.359784.
Making request for The Isle Of Skye Brewing Co Ltd, 57.5867774, -6.3777137.
Making request for Brauhaus Lemke am Hackeschen Markt, 52.5232

In [165]:
print(all_yelp_result)

{'businesses': [{'id': '05wBSwtKBCad4B9GpzXY_g', 'alias': 'yazoo-brewing-company-nashville-3', 'name': 'Yazoo Brewing Company', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7wQHeh4f4xafjjtqCPsFAw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/yazoo-brewing-company-nashville-3?adjust_creative=aYqGPnh-SCvFYwUiKYbPng&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=aYqGPnh-SCvFYwUiKYbPng', 'review_count': 356, 'categories': [{'alias': 'breweries', 'title': 'Breweries'}], 'rating': 4.5, 'coordinates': {'latitude': 36.15113, 'longitude': -86.78199}, 'transactions': [], 'price': '$', 'location': {'address1': '910 Division St', 'address2': '', 'address3': '', 'city': 'Nashville', 'zip_code': '37203', 'country': 'US', 'state': 'TN', 'display_address': ['910 Division St', 'Nashville, TN 37203']}, 'phone': '+16158914649', 'display_phone': '(615) 891-4649', 'distance': 3.556319399733787}], 'total': 19, 'region': {'center': {'longitude': -86.782068, 'lat

In [167]:
all_yelp_df = pd.DataFrame({'Brewery': all_yelp_name_list, 'Rating': all_yelp_rating_list, 'Review Count': all_yelp_count_list})
all_yelp_df = all_yelp_df.reset_index()
all_yelp_df

,index,Brewery,Rating,Review Count
0,0,Vecchio Arzere,4,1
1,1,Caldera Brewery & Restaurant,4,397
2,2,Broad Ripple Brewpub,3.5,313
3,3,Moon River Brewing Company,3,1162
4,4,Snowdonia Park Brewpub,4,1
5,5,Public Coast Brewing Company,3.5,209
6,6,San Francisco Brewing,3.5,102
7,7,Yazoo Brewing Company,4.5,356
8,8,7 Seas Brewing,4,132
9,9,Calapooia Brewing Company,4,147


In [ ]:
merge_all_df = pd.merge(all_df, all_yelp_df, on='index', how='outer')
merge_all_df = merge_all_df.rename(columns={'Brewery_x': 'Google Name', 'Rating_x': 'Google Rating', 'Brewery_y': 'Yelp Name', 'Rating_y': 'Yelp Rating', 'Review Count': 'Yelp Review Count'})
merge_all_df.to_csv('merge_top_df.csv')
merge_all_df

In [6]:
beer_count = df_beer_reviews.groupby(['beer_name']).count()
beer_count = beer_count[['review_overall']]
beer_count = beer_count.reset_index()
beer_count

,beer_name,review_overall
0,! (Old Ale),1
1,"""100""",5
2,"""100"" Pale Ale",1
3,"""12"" Belgian Golden Strong Ale",2
4,"""33"" Export",3
5,"""4"" Horse Oatmeal Stout",2
6,"""400"" Ale",18
7,"""50"" Golden Anniversary Lager",1
8,"""76"" Anniversary Ale",3
9,"""76"" Anniversary Ale With English Hops",1


In [9]:
beer_df=pd.DataFrame(beer.mean())
beer_df=beer_df.reset_index()

beer_merged_data = pd.merge(beer_df, beer_count, on='beer_name', how='outer')
beer_merged_data = beer_merged_data.rename(columns={'review_overall_x': 'average_review', 'review_overall_y': 'review_count'})
beer_merged_data.head()

,beer_name,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,review_count
0,! (Old Ale),NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.0,4.5,4.00,4.000000,4.5,8.2,57645.0,1
1,"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.0,4.0,4.00,4.000000,4.4,10.0,49236.0,5
2,"""100"" Pale Ale",Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.0,4.0,3.50,4.000000,4.0,6.6,50696.0,1
3,"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.0,3.5,4.25,4.750000,4.5,9.0,44353.0,2
4,"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.0,3.0,3.00,3.166667,3.0,4.8,30080.0,3


In [10]:
beer_sorted_data = beer_merged_data.sort_values('review_count', ascending=False)
beer_sorted_data.head()

,beer_name,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,review_count
944,90 Minute IPA,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093.0,3290
30531,India Pale Ale,Altes Tramdepot,American IPA,4558.0,1.320183e+09,4.000000,4.000000,3.500000,3.500000,3.500000,5.5,74771.0,3130
30628,India Pale Ale,San Pedro Brewing Company,American IPA,1018.0,1.202529e+09,3.750000,3.000000,3.500000,3.000000,3.500000,11.0,27581.0,3130
30621,India Pale Ale,Redhook Ale Brewery,English India Pale Ale (IPA),18134.0,1.272264e+09,4.000000,4.000000,4.000000,4.000000,4.500000,NaN,57849.0,3130
30622,India Pale Ale,Redwood Curtain Brewing Company,American IPA,22065.0,1.300587e+09,4.000000,4.000000,4.000000,4.000000,4.000000,6.4,67359.0,3130


In [11]:
bins = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
review_count_slices = ['1', '2', '3', '4-5', '6-8', '9-13', '14-21', '22-34', '35-55', '56-89', '90-144', '145-233', '234-377', '378-610', '611-987', '988-1597', '1598-2584', '2585 and up']
beer_sorted_data['review_count_slices'] = pd.cut(beer_sorted_data['review_count'], bins, labels=review_count_slices)
beer_sorted_data.head()

,beer_name,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,review_count,review_count_slices
944,90 Minute IPA,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093.0,3290,2585 and up
30531,India Pale Ale,Altes Tramdepot,American IPA,4558.0,1.320183e+09,4.000000,4.000000,3.500000,3.500000,3.500000,5.5,74771.0,3130,2585 and up
30628,India Pale Ale,San Pedro Brewing Company,American IPA,1018.0,1.202529e+09,3.750000,3.000000,3.500000,3.000000,3.500000,11.0,27581.0,3130,2585 and up
30621,India Pale Ale,Redhook Ale Brewery,English India Pale Ale (IPA),18134.0,1.272264e+09,4.000000,4.000000,4.000000,4.000000,4.500000,NaN,57849.0,3130,2585 and up
30622,India Pale Ale,Redwood Curtain Brewing Company,American IPA,22065.0,1.300587e+09,4.000000,4.000000,4.000000,4.000000,4.000000,6.4,67359.0,3130,2585 and up


In [77]:
hoover_101_b = beer_sorted_data[beer_sorted_data['brewery_name'] == '''Hoover's 101 Cafe''']
hoover_101_b
#https://www.beeradvocate.com/beer/profile/15705/40529/
#https://www.yelp.com/biz/hoovers-101-cafe-atascadero

,beer_name,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,review_count,review_count_slices
30579,India Pale Ale,Hoover's 101 Cafe,American IPA,15705.0,1.200176e+09,4.00,4.00,5.00,3.50,4.00,NaN,40529.0,3130,2585 and up
44306,Pale Ale,Hoover's 101 Cafe,American Pale Ale (APA),15705.0,1.193383e+09,4.00,3.50,3.25,4.00,3.75,4.5,38069.0,2500,1598-2584
46498,Porter,Hoover's 101 Cafe,American Porter,15705.0,1.200176e+09,4.00,4.00,4.00,4.00,4.00,NaN,40531.0,1595,988-1597
2439,Amber Ale,Hoover's 101 Cafe,American Amber / Red Ale,15705.0,1.200176e+09,3.50,3.50,4.00,3.00,3.00,NaN,40533.0,555,378-610
9237,Blueberry Wheat,Hoover's 101 Cafe,Hefeweizen,15705.0,1.203180e+09,4.00,2.50,3.50,2.50,2.50,NaN,41030.0,88,56-89
2781,American Wheat,Hoover's 101 Cafe,American Pale Wheat Ale,15705.0,1.193681e+09,3.75,3.75,3.75,3.75,3.75,4.2,38209.0,49,35-55


In [12]:
review_1 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1', :]
review_1_count = review_1['review_count'].count()
review_2 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2', :]
review_2_count = review_2['review_count'].count()
review_3 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '3', :]
review_3_count = review_3['review_count'].count()
review_4_5 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '4-5', :]
review_4_5_count = review_4_5['review_count'].count()
review_6_8 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '6-8', :]
review_6_8_count = review_6_8['review_count'].count()
review_9_13 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '9-13', :]
review_9_13_count = review_9_13['review_count'].count()
review_14_21 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '14-21', :]
review_14_21_count = review_14_21['review_count'].count()
review_22_34 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '22-34', :]
review_22_34_count = review_22_34['review_count'].count()
review_35_55 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '35-55', :]
review_35_55_count = review_35_55['review_count'].count()
review_56_89 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '56-89', :]
review_56_89_count = review_56_89['review_count'].count()
review_90_144 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '90-144', :]
review_90_144_count = review_90_144['review_count'].count()
review_145_233 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '145-233', :]
review_145_233_count = review_145_233['review_count'].count()
review_234_377 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '234-377', :]
review_234_377_count = review_234_377['review_count'].count()
review_378_610 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '378-610', :]
review_378_610_count = review_378_610['review_count'].count()
review_611_987 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '611-987', :]
review_611_987_count = review_611_987['review_count'].count()
review_988_1597 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '988-1597', :]
review_988_1597_count = review_988_1597['review_count'].count()
review_1598_2584 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1598-2584', :]
review_1598_2594_count = review_1598_2584['review_count'].count()
review_2585_andup = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2585 and up', :]
review_2585_andup_count = review_2585_andup['review_count'].count()



review_count_df = pd.DataFrame({'review_count': [review_1_count, review_2_count, review_3_count, review_4_5_count, review_6_8_count, review_9_13_count, review_14_21_count, review_22_34_count, review_35_55_count, review_56_89_count, review_90_144_count, review_145_233_count, review_234_377_count, review_378_610_count, review_611_987_count, review_988_1597_count, review_1598_2594_count, review_2585_andup_count]})
review_count_df = review_count_df.rename(index={0 : '1', 1 : '2', 2 : '3', 3 : '4-5', 4 : '6-8', 5 : '9-13', 6 : '14-21', 7 : '22-34', 8 : '35-55',
                                                9: '56-89', 10: '90-144', 11: '145-233', 12: '234-377', 13: '378-610', 14: '611-987', 15: '988-1597', 16: '1598-2584', 17: '2585 and up'})
review_count_df

,review_count
1,18908
2,8779
3,5241
4-5,5981
6-8,4732
9-13,3985
14-21,3319
22-34,2698
35-55,2446
56-89,2033


In [13]:
top_list = list(review_2585_andup['brewery_name'])
len(top_list), top_list

review_2585_andup['brewery_name'].value_counts()

Sierra Nevada Brewing Co.                      2
Stone Brewing Co.                              2
Okanagan Spring Brewery                        1
Redwood Curtain Brewing Company                1
Chez Gambrinus                                 1
Mogollon Brewing Company                       1
Ross Valley Brewing Company                    1
California Brewing Company                     1
Laht Neppur Brewing Company                    1
HaandBryggeriet                                1
Walla Walla Brewers Inc.                       1
Doppelganger's Bar & Grill                     1
Shipwrecked Brew Pub, Restaurant and Inn       1
Bridge Brew Works LLC                          1
Manayunk Brewery and Restaurant                1
Coddington Brewing Co.                         1
King & Barnes Ltd.                             1
Three Rivers Eatery And Brewhouse              1
Boxcar Brewing Co.                             1
Davidson Brothers Restaurant & Brewery         1
The Lion Brewery Res

In [14]:
top_set = set(top_list)
len(top_set), top_set

(145,
 {'5280 Roadhouse and Brewery',
  'Alcatraz Brewing Co.',
  'Altes Tramdepot',
  'Amundsen Bryggeri & Spiseri',
  "Beermann's Beerwerks",
  "Bell's Brewery, Inc.",
  'Belmont Brewing Company',
  'Bent River Brewing Company',
  'Big Rock Brewery',
  'Bitter Root Brewing',
  'Blue Frog Grog & Grill',
  "Bluegrass Brewing Co. - East St. Matthew's",
  'Boxcar Brewing Co.',
  'Brasserie Trois Dames',
  'Brewerkz Riverside Point',
  'Brewsters Brewing Company & Restaurant - Oliver Square',
  'Bricktown Brewery',
  'Bridge Brew Works LLC',
  'Broadway Brewery & Restaurant',
  'Bryggeri Skovlyst',
  "Bube's Brewery",
  'Burris Brewing',
  'California Brewing Company',
  'Carlyle Brewing Company',
  'Chez Gambrinus',
  "Choc Beer Co. / Pete's Place",
  'Climax Brewing Company',
  'Coddington Brewing Co.',
  'Court Avenue Brewing Company',
  'Dark Isle Brewery',
  'Davidson Brothers Restaurant & Brewery',
  'Dogfish Head Brewery',
  "Doppelganger's Bar & Grill",
  'Double Take Brewing Comp

In [15]:
#url = "https://api.yelp.com/v3/businesses/search?term="

#top_response = requests.get(url + "Sierra Nevada")
#top_response


In [16]:




API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/'
def get_business(business_id):
    business_path = BUSINESS_PATH + business_id
    url = API_HOST + business_path + '/reviews'
    headers = {'Authorization': f"Bearer {API_KEY}"}

    response = requests.get(url, headers=headers)

    return response.json()
results = get_business('la-taqueria-san-francisco-2')
pprint(results)

{'possible_languages': ['fr', 'en', 'de', 'it', 'ja', 'es'],
 'reviews': [{'id': 'K-W5Nf2K_yhRH1g_h80CHg',
              'rating': 5,
              'text': 'Oh man, La Taqueria was awesome! From the moment we got '
                      'to San Francisco, everyone was telling us about this '
                      'place. This restaurant is located in the...',
              'time_created': '2018-10-30 15:29:22',
              'url': 'https://www.yelp.com/biz/la-taqueria-san-francisco-2?hrid=K-W5Nf2K_yhRH1g_h80CHg&adjust_creative=aYqGPnh-SCvFYwUiKYbPng&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=aYqGPnh-SCvFYwUiKYbPng',
              'user': {'id': 'fUhpS4c7rMxmn_niVhVbNA',
                       'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/RK2SWFEq1G3Mp5NiqVfYUg/o.jpg',
                       'name': 'Maria U.',
                       'profile_url': 'https://www.yelp.com/user_details?userid=fUhpS4c7rMxmn_niVhVbNA'}},
             {'id': '-FIg64xTSuvgn3

In [17]:
API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/'
def get_business(business_id):
    business_path = BUSINESS_PATH + business_id
    url = API_HOST + business_path
    headers = {'Authorization': f"Bearer {API_KEY}"}

    response = requests.get(url, headers=headers)

    return response.json()
results = get_business('la-taqueria-san-francisco-2')
pprint(results)

{'alias': 'la-taqueria-san-francisco-2',
 'categories': [{'alias': 'mexican', 'title': 'Mexican'}],
 'coordinates': {'latitude': 37.750883, 'longitude': -122.418123},
 'display_phone': '(415) 285-7117',
 'hours': [{'hours_type': 'REGULAR',
            'is_open_now': True,
            'open': [{'day': 2,
                      'end': '2045',
                      'is_overnight': False,
                      'start': '1100'},
                     {'day': 3,
                      'end': '2045',
                      'is_overnight': False,
                      'start': '1100'},
                     {'day': 4,
                      'end': '2045',
                      'is_overnight': False,
                      'start': '1100'},
                     {'day': 5,
                      'end': '2045',
                      'is_overnight': False,
                      'start': '1100'},
                     {'day': 6,
                      'end': '1945',
                      'is_overnight': Fals

In [18]:
term = "Stone Brewing"
location = "escondido"
SEARCH_LIMIT = 3
category = "breweries"
sort = "review_count"

url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'categories': category,
        'sort_by': sort
    }
    
API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/search'

business_path = BUSINESS_PATH
url = API_HOST + business_path
headers = {'Authorization': f"Bearer {API_KEY}"}

response = requests.get(url, headers=headers, params=url_params)

results = response.json()

pprint(results)

{'businesses': [{'alias': 'stone-brewing-world-bistro-and-gardens-liberty-station-san-diego-2',
                 'categories': [{'alias': 'newamerican',
                                 'title': 'American (New)'},
                                {'alias': 'breweries', 'title': 'Breweries'},
                                {'alias': 'bars', 'title': 'Bars'}],
                 'coordinates': {'latitude': 32.740505,
                                 'longitude': -117.211664},
                 'display_phone': '(619) 269-2100',
                 'distance': 45644.87124795605,
                 'id': '7o_XZUi0pBQldtxOgbjSwA',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/RRsAIPccWJeueLBev582yQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '2816 Historic Decatur Rd',
                              'address2': 'Ste 116',
                              'address3': '',
                              'city': 'San Diego',
               

In [19]:
brewery = "alpha brewing"
put = "textquery"
field = ["id", "name", "rating", "geometry/location"]
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
params = {
    "input": brewery,
    "key": gkey,
    "inputtype": put,
    "fields": ["name", "id"]
}
google_response = requests.get(base_url, params=params)
print(response.url)

https://api.yelp.com/v3/businesses/search?term=Stone%2BBrewing&location=escondido&limit=3&categories=breweries&sort_by=review_count


In [20]:
brewery = "climax brewing company"
put = "textquery"
field = "id,name,rating,geometry/location"
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
params = {
    "input": brewery,
    "key": gkey,
    "inputtype": put,
    "fields": field
}
google_response = requests.get(base_url, params=params)
print(response.url)

https://api.yelp.com/v3/businesses/search?term=Stone%2BBrewing&location=escondido&limit=3&categories=breweries&sort_by=review_count


In [21]:
places_data = google_response.json()

pprint(places_data)
#print(json.dumps(places_data, indent=4, sort_keys=True))

{'candidates': [{'geometry': {'location': {'lat': 40.659725,
                                           'lng': -74.28300399999999}},
                 'id': '90ce939b553ba980b179f58384274f260eeec685',
                 'name': 'Climax Brewing Co',
                 'rating': 4.9}],
 'debug_log': {'line': []},
 'status': 'OK'}


In [57]:
latitude = places_data['candidates'][0]['geometry']['location']['lat']
longitude = places_data['candidates'][0]['geometry']['location']['lng']
name = places_data['candidates'][0]['name']
latitude, longitude, name

(40.659725, -74.28300399999999, 'Climax Brewing Co')

In [159]:
latitude = "45.5509"
longitude = "11.8285"

SEARCH_LIMIT = 1
category = "breweries"
sort = "distance"

url_params = {
        'longitude': longitude,
        'latitude': latitude,
        'limit': SEARCH_LIMIT,
        'categories': category,
        'sort_by': sort
    }
    
API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/search'

business_path = BUSINESS_PATH
url = API_HOST + business_path
headers = {'Authorization': f"Bearer {API_KEY}"}
yelp_response = requests.get(url, headers=headers, params=url_params)
print(yelp_response.url)
print(yelp_response)

https://api.yelp.com/v3/businesses/search?longitude=11.8285&latitude=45.5509&limit=1&categories=breweries&sort_by=distance
<Response [200]>


In [162]:



yelp_results = yelp_response.json()

pprint(yelp_results)

{'businesses': [{'alias': 'vecchio-arzere-piazzola-sul-brenta',
                 'categories': [{'alias': 'pubs', 'title': 'Pubs'},
                                {'alias': 'breweries', 'title': 'Breweries'}],
                 'coordinates': {'latitude': 45.5285110473633,
                                 'longitude': 11.7735614776611},
                 'display_phone': '+39 049 960 0020',
                 'distance': 4922.542053095365,
                 'id': '9eF3j63VeF8bzEi9TF_TlQ',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/4WnTptCSqWBJOjcZZVLseQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': 'Via Boschi 26',
                              'address2': '',
                              'address3': '',
                              'city': 'Piazzola sul Brenta',
                              'country': 'IT',
                              'display_address': ['Via Boschi 26',
                                             

In [161]:
name = yelp_results['businesses'][0]['name']
rating = yelp_results['businesses'][0]['rating']
review_count = yelp_results['businesses'][0]['review_count']
name, rating, review_count

('Vecchio Arzere', 4.0, 1)

In [26]:
top_set

{'5280 Roadhouse and Brewery',
 'Alcatraz Brewing Co.',
 'Altes Tramdepot',
 'Amundsen Bryggeri & Spiseri',
 "Beermann's Beerwerks",
 "Bell's Brewery, Inc.",
 'Belmont Brewing Company',
 'Bent River Brewing Company',
 'Big Rock Brewery',
 'Bitter Root Brewing',
 'Blue Frog Grog & Grill',
 "Bluegrass Brewing Co. - East St. Matthew's",
 'Boxcar Brewing Co.',
 'Brasserie Trois Dames',
 'Brewerkz Riverside Point',
 'Brewsters Brewing Company & Restaurant - Oliver Square',
 'Bricktown Brewery',
 'Bridge Brew Works LLC',
 'Broadway Brewery & Restaurant',
 'Bryggeri Skovlyst',
 "Bube's Brewery",
 'Burris Brewing',
 'California Brewing Company',
 'Carlyle Brewing Company',
 'Chez Gambrinus',
 "Choc Beer Co. / Pete's Place",
 'Climax Brewing Company',
 'Coddington Brewing Co.',
 'Court Avenue Brewing Company',
 'Dark Isle Brewery',
 'Davidson Brothers Restaurant & Brewery',
 'Dogfish Head Brewery',
 "Doppelganger's Bar & Grill",
 'Double Take Brewing Company',
 'Downtown Brewing Co.',
 'Draper 

In [76]:
hoover_101 = df_beer_reviews[df_beer_reviews['brewery_name'] == "Hoover's 101 Cafe"]
hoover_101

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
828494,15705,Hoover's 101 Cafe,1200176330,3.5,3.5,4.0,barleywinefiend,American Amber / Red Ale,3.0,3.0,Amber Ale,NaN,40533
836333,15705,Hoover's 101 Cafe,1200176013,4.0,4.0,5.0,barleywinefiend,American IPA,3.5,4.0,India Pale Ale,NaN,40529
836437,15705,Hoover's 101 Cafe,1200175691,3.5,3.5,3.5,barleywinefiend,American Pale Wheat Ale,3.5,3.5,American Wheat,4.2,38209
836657,15705,Hoover's 101 Cafe,1187185603,4.0,4.0,4.0,bradybeer,American Pale Wheat Ale,4.0,4.0,American Wheat,4.2,38209
836723,15705,Hoover's 101 Cafe,1200176188,4.0,4.0,4.0,barleywinefiend,American Porter,4.0,4.0,Porter,NaN,40531
836796,15705,Hoover's 101 Cafe,1200175849,4.0,4.0,3.5,barleywinefiend,American Pale Ale (APA),4.0,4.0,Pale Ale,4.5,38069
837017,15705,Hoover's 101 Cafe,1186590840,4.0,3.0,3.0,bradybeer,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,38069
837248,15705,Hoover's 101 Cafe,1203179811,4.0,2.5,3.5,barleywinefiend,Hefeweizen,2.5,2.5,Blueberry Wheat,NaN,41030


In [15]:
 
put = "textquery"                                
field = "id,name,rating,geometry/location"
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
lat_list = []
lng_list = []
brew_list = []
rating_list = []
response_list = []
for brewery in top_set:
    params = {
    "input": brewery,
    "key": gkey,
    "inputtype": put,
    "fields": field
}
    print(f"Making request for {brewery}.")
    top_response = requests.get(base_url, params=params)
    result = top_response.json()
    response_list.append(top_response.json())
    
    try:
        lat_list.append(result['candidates'][0]['geometry']['location']['lat'])
        lng_list.append(result['candidates'][0]['geometry']['location']['lng'])
        brew_list.append(result['candidates'][0]['name'])
        rating_list.append(result['candidates'][0]['rating'])
    except:
        print(f"No result for {brewery}.")
        lat_list.append('No Google Result')
        lng_list.append('No Google Result')
        brew_list.append(f'No Google Result for {brewery}')
        rating_list.append('No Google Result')
    
print(top_response.url)


Making request for Hoover's 101 Cafe.
Making request for Dark Isle Brewery.
No result for Dark Isle Brewery.
Making request for Double Take Brewing Company.
No result for Double Take Brewing Company.
Making request for LaConner Brewing Company.
Making request for Richbrau Brewing Company.
Making request for Emmett's Brewing Company.
No result for Emmett's Brewing Company.
Making request for Brewerkz Riverside Point.
Making request for Monterey Coast Brewing.
Making request for Shepherd Neame Ltd.
Making request for The Pump Room.
Making request for Granville Island Brewery.
Making request for The Olde Stone Brewing Company.
Making request for Mad Boar Restaurant & Brewery.
Making request for Mordue Brewery.
Making request for Yokohama Beer.
Making request for Manayunk Brewery and Restaurant.
Making request for Three Rivers Eatery And Brewhouse.
Making request for Walla Walla Brewers Inc..
Making request for Innis & Gunn.
No result for Innis & Gunn.
Making request for Laht Neppur Brewin

In [16]:
len(brew_list), len(lat_list), len(lng_list), len(rating_list)

(145, 145, 145, 145)

In [22]:
top_df = pd.DataFrame({'Brewery': brew_list, 'Latitude': lat_list, 'Longitude': lng_list, 'Rating': rating_list})
top_df = top_df.reset_index()

top_df

,index,Brewery,Latitude,Longitude,Rating
0,0,Stanford Blood Draw at Hoover Pavilion,37.4415,-122.169,4.8
1,1,No Google Result for Dark Isle Brewery,No Google Result,No Google Result,No Google Result
2,2,No Google Result for Double Take Brewing Company,No Google Result,No Google Result,No Google Result
3,3,La Conner Brewing Co,48.3918,-122.496,4.4
4,4,Richbrau Brewing,37.5314,-77.426,0
5,5,No Google Result for Emmett's Brewing Company,No Google Result,No Google Result,No Google Result
6,6,Brewerkz Riverside Point,1.28923,103.844,4.3
7,7,Monterey Coast Brewing,36.676,-121.655,4.2
8,8,Shepherd Neame Brewery,51.3167,0.891625,4.4
9,9,The Pump Room,33.7873,-117.869,4.4


In [18]:
test_name = top_df.iloc[0,0]
test_latitude = top_df.iloc[0,1]
test_longitude = top_df.iloc[0,2]


test_latitude, test_longitude, test_name

('Stanford Blood Draw at Hoover Pavilion', 37.44151069999999, 0)

In [72]:

SEARCH_LIMIT = 1
category = "breweries"
sort = "distance"

url_params = {
        'longitude': test_longitude,
        'latitude': test_latitude,
        'limit': SEARCH_LIMIT,
        'categories': category,
        'sort_by': sort
    }
    
API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/search'

business_path = BUSINESS_PATH
url = API_HOST + business_path
headers = {'Authorization': f"Bearer {API_KEY}"}
yelp_test_response = requests.get(url, headers=headers, params=url_params)
print(yelp_test_response.url)
print(test_longitude)

https://api.yelp.com/v3/businesses/search?longitude=-122.1685813&latitude=37.44151069999999&limit=1&categories=breweries&sort_by=distance
-122.1685813


In [73]:
yelp_test_results = yelp_test_response.json()

pprint(yelp_test_results)

{'businesses': [{'alias': 'dan-gordons-palo-alto',
                 'categories': [{'alias': 'gastropubs', 'title': 'Gastropubs'},
                                {'alias': 'breweries', 'title': 'Breweries'},
                                {'alias': 'bbq', 'title': 'Barbeque'}],
                 'coordinates': {'latitude': 37.4433619437366,
                                 'longitude': -122.161041240946},
                 'display_phone': '(650) 324-1960',
                 'distance': 696.7732861937759,
                 'id': '_5Grk2YhMTs9lKpFQUXuwQ',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/0td8jymn1a5_KPengwW0-Q/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '640 Emerson St',
                              'address2': '',
                              'address3': '',
                              'city': 'Palo Alto',
                              'country': 'US',
                              'display_address': ['6

In [69]:
test_name = yelp_test_results['businesses'][0]['name']
test_rating = yelp_test_results['businesses'][0]['rating']
test_review_count = yelp_test_results['businesses'][0]['review_count']
test_name, test_rating, test_review_count

('Climax Brewing Co', 4.0, 29)

In [19]:
yelp_name_list = []
yelp_rating_list  = []
yelp_count_list = []
SEARCH_LIMIT = 1
category = "breweries"
sort = "distance"
yelp_response_list = []

    


for index,row in top_df.iterrows():
    API_HOST = 'https://api.yelp.com'
    BUSINESS_PATH = '/v3/businesses/search'
    business_path = BUSINESS_PATH
    url = API_HOST + business_path
    headers = {'Authorization': f"Bearer {API_KEY}"}
    url_params = {
        'longitude': row['Longitude'],
        'latitude': row['Latitude'],
        'limit': SEARCH_LIMIT,
        'categories': category,
        'sort_by': sort
    }
    print(f"Making request for {row['Brewery']}.")
    yelp_top_response = requests.get(url, headers=headers, params=url_params)
    result = yelp_top_response.json()
    yelp_response_list.append(yelp_top_response.json())
    
    try:
        yelp_name_list.append(result['businesses'][0]['name'])
        yelp_rating_list.append(result['businesses'][0]['rating'])
        yelp_count_list.append(result['businesses'][0]['review_count'])
        
    except:
        print(f"No result for {row['Brewery']}.")
        yelp_name_list.append(f"No Yelp Result for {row['Brewery']}")
        yelp_rating_list.append("No Yelp Result")
        yelp_count_list.append("No Yelp Result")




Making request for Stanford Blood Draw at Hoover Pavilion.
Making request for No Google Result for Dark Isle Brewery.
No result for No Google Result for Dark Isle Brewery.
Making request for No Google Result for Double Take Brewing Company.
No result for No Google Result for Double Take Brewing Company.
Making request for La Conner Brewing Co.
Making request for Richbrau Brewing.
Making request for No Google Result for Emmett's Brewing Company.
No result for No Google Result for Emmett's Brewing Company.
Making request for Brewerkz Riverside Point.
Making request for Monterey Coast Brewing.
Making request for Shepherd Neame Brewery.
Making request for The Pump Room.
Making request for Granville Island Brewing.
Making request for Olde Stone Brewing Company.
Making request for Mad Boar Restaurant.
Making request for Mordue Brewery.
Making request for World of Beer Museum Yokohama.
Making request for Manayunk Brewing Company.
No result for Manayunk Brewing Company.
Making request for Thre

Making request for No Google Result for Land Beer.
No result for No Google Result for Land Beer.
Making request for No Google Result for Sierra Nevada Brewing Co..
No result for No Google Result for Sierra Nevada Brewing Co..
Making request for Beermann's Beerwerks Brewery.
Making request for Liberty Brewery and Grill.
Making request for McMenamins Queen Anne.
Making request for Bent River Brewing Company.
Making request for Brewsters.
Making request for No Google Result for Iron Horse Brewery.
No result for No Google Result for Iron Horse Brewery.
Making request for Iron House Brewery.
Making request for Zero Gravity Brewpub at American Flatbread.
Making request for No Google Result for Weyerbacher Brewing Co..
No result for No Google Result for Weyerbacher Brewing Co..
Making request for Wagner Valley Brewing Company.
Making request for Powerhouse Brewing Company.
Making request for Stensbogaard Bryghus.
Making request for No Google Result for Dogfish Head Brewery.
No result for No G

In [31]:
print(yelp_top_response.url)

https://api.yelp.com/v3/businesses/search?longitude=-121.6550822&latitude=36.675964&limit=1&categories=breweries&sort_by=distance


In [46]:
yelp_name_list

[]

In [40]:
yelp_response_list

[{'businesses': [{'id': '_5Grk2YhMTs9lKpFQUXuwQ',
    'alias': 'dan-gordons-palo-alto',
    'name': "Dan Gordon's",
    'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/0td8jymn1a5_KPengwW0-Q/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/dan-gordons-palo-alto?adjust_creative=aYqGPnh-SCvFYwUiKYbPng&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=aYqGPnh-SCvFYwUiKYbPng',
    'review_count': 295,
    'categories': [{'alias': 'gastropubs', 'title': 'Gastropubs'},
     {'alias': 'breweries', 'title': 'Breweries'},
     {'alias': 'bbq', 'title': 'Barbeque'}],
    'rating': 3.5,
    'coordinates': {'latitude': 37.4433619437366,
     'longitude': -122.161041240946},
    'transactions': ['pickup'],
    'price': '$$',
    'location': {'address1': '640 Emerson St',
     'address2': '',
     'address3': '',
     'city': 'Palo Alto',
     'zip_code': '94301',
     'country': 'US',
     'state': 'CA',
     'display_address': ['640 Emerson St', 'Palo 

In [24]:
yelp_top_df = pd.DataFrame({'Brewery': yelp_name_list, 'Rating': yelp_rating_list, 'Review Count': yelp_count_list})
yelp_top_df = yelp_top_df.reset_index()
yelp_top_df

,index,Brewery,Rating,Review Count
0,0,Dan Gordon's,3.5,295
1,1,No Yelp Result for No Google Result for Dark I...,No Yelp Result,No Yelp Result
2,2,No Yelp Result for No Google Result for Double...,No Yelp Result,No Yelp Result
3,3,La Conner Brewing,3.5,226
4,4,Lickinghole Creek Brew Pub,4,6
5,5,No Yelp Result for No Google Result for Emmett...,No Yelp Result,No Yelp Result
6,6,District 10 Fish Chips Burger Bar,4,1
7,7,Monterey Coast Brewing Company,3.5,223
8,8,Shepherd Neame,3.5,3
9,9,Chapman Crafted Beer,4.5,101


In [42]:
merge_top_df = pd.merge(top_df, yelp_top_df, on='index', how='outer')
merge_top_df = merge_top_df.rename(columns={'Brewery_x': 'Google Name', 'Rating_x': 'Google Rating', 'Brewery_y': 'Yelp Name', 'Rating_y': 'Yelp Rating', 'Review Count': 'Yelp Review Count'})

merge_top_df = merge_top_df[~merge_top_df['Latitude'].astype(str).str.contains('No Google Result')]
merge_top_df = merge_top_df[~merge_top_df['Yelp Rating'].astype(str).str.contains('No Yelp Result')]
merge_top_df['Average Rating'] = (merge_top_df['Google Rating'] + merge_top_df['Yelp Rating']) / 2 

merge_top_df.to_csv('merge_top_df.csv')
merge_top_df

,index,Google Name,Latitude,Longitude,Google Rating,Yelp Name,Yelp Rating,Yelp Review Count,Average Rating
0,0,Stanford Blood Draw at Hoover Pavilion,37.4415,-122.169,4.8,Dan Gordon's,3.5,295,4.15
3,3,La Conner Brewing Co,48.3918,-122.496,4.4,La Conner Brewing,3.5,226,3.95
4,4,Richbrau Brewing,37.5314,-77.426,0,Lickinghole Creek Brew Pub,4,6,2
6,6,Brewerkz Riverside Point,1.28923,103.844,4.3,District 10 Fish Chips Burger Bar,4,1,4.15
7,7,Monterey Coast Brewing,36.676,-121.655,4.2,Monterey Coast Brewing Company,3.5,223,3.85
8,8,Shepherd Neame Brewery,51.3167,0.891625,4.4,Shepherd Neame,3.5,3,3.95
9,9,The Pump Room,33.7873,-117.869,4.4,Chapman Crafted Beer,4.5,101,4.45
10,10,Granville Island Brewing,49.2705,-123.136,4.3,Granville Island Brewing,3.5,313,3.9
11,11,Olde Stone Brewing Company,44.3053,-78.3198,4.5,The Publican House Brewery,3.5,3,4
12,12,Mad Boar Restaurant,34.7569,-77.9478,4,Salty Turtle Beer Company,4.5,27,4.25


In [39]:
locations = merge_top_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = merge_top_df["Average Rating"].astype(float)

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
response_df = pd.DataFrame(response_list)
response_df

,candidates,debug_log,error_message,status
0,[{'geometry': {'location': {'lat': 37.44151069...,{'line': []},NaN,OK
1,"[{'geometry': {'location': {'lat': 41.5853134,...",{'line': []},NaN,OK
2,"[{'geometry': {'location': {'lat': 38.0940901,...",{'line': []},NaN,OK
3,"[{'geometry': {'location': {'lat': 40.0225428,...",{'line': []},NaN,OK
4,[{'geometry': {'location': {'lat': 48.65200859...,{'line': []},NaN,OK
5,"[{'geometry': {'location': {'lat': 42.7727232,...",{'line': []},NaN,OK
6,"[{'geometry': {'location': {'lat': 41.254724, ...",{'line': []},NaN,OK
7,[],{'line': []},NaN,ZERO_RESULTS
8,"[{'geometry': {'location': {'lat': 42.8234105,...",{'line': []},NaN,OK
9,[],{'line': []},NaN,ZERO_RESULTS


In [35]:
response_df.to_csv('top_set_response.csv')

In [36]:
beer_df.head()
# beer_df.to_csv('Beer Data Groupby.csv')

,beer_name,brewery_name,beer_style,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
0,! (Old Ale),NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.0,4.5,4.00,4.000000,4.5,8.2,57645.0
1,"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.0,4.0,4.00,4.000000,4.4,10.0,49236.0
2,"""100"" Pale Ale",Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.0,4.0,3.50,4.000000,4.0,6.6,50696.0
3,"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.0,3.5,4.25,4.750000,4.5,9.0,44353.0
4,"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.0,3.0,3.00,3.166667,3.0,4.8,30080.0
